# csv 파일을 불러 오는것 부터 문제가 있었다.
    한글코딩을 위해서 encoding='CP949' 를 적어주었다.
    파일에 대한 설명이 맨 위에 적혀 있었는데, 데이터 프레임과 맞지 않아서 읽어 오기가 힘들었다. 
    직접 csv파일 윗부분 몇 줄을 내가 삭제했는데(데이터 자체를 지운것은 아니고, 파일에 있는 주석 10줄 정도 삭제), 이렇게 해도 되는 것인지 모르겠다.

In [68]:
# 필요한 모듈 임포트

import numpy as np # 선형대수 
import pandas as pd # 데이터 프레임
%matplotlib inline 
# 그래프 쥬피터 노트북 위에 그려지게끔 
import seaborn as sns # 데이터 시각화 
import matplotlib.pyplot as plt # 데이터 시각화 

    데이터 출처 : https://rt.molit.go.kr
    기간 : 2017년 01월 01일 ~ 2020년 12월 31일
    범위 : 서울시 아파트 계약일을 기준으로 한 아파트 매매 거래 금액 

In [69]:
apart17 = pd.read_csv("data/2017아파트실거래가.csv", encoding='CP949')
apart18 = pd.read_csv("data/2018아파트실거래가.csv", encoding='CP949')
apart19 = pd.read_csv("data/2019아파트실거래가.csv", encoding='CP949')
apart20 = pd.read_csv("data/2020아파트실거래가.csv", encoding='CP949')

In [70]:
# 따로 저장되어 있는 자료를 하나로 합치기
apart = pd.concat([apart17,apart18,apart19,apart20])

In [71]:
# 기본 내용 요약
apart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345993 entries, 0 to 84225
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       345993 non-null  object 
 1   번지        345806 non-null  object 
 2   본번        345939 non-null  float64
 3   부번        345939 non-null  float64
 4   단지명       345993 non-null  object 
 5   전용면적(㎡)   345993 non-null  float64
 6   계약년월      345993 non-null  int64  
 7   계약일       345993 non-null  int64  
 8   거래금액(만원)  345993 non-null  object 
 9   층         345993 non-null  int64  
 10  건축년도      345993 non-null  int64  
 11  도로명       345993 non-null  object 
 12  해제사유발생일   3038 non-null    float64
dtypes: float64(4), int64(4), object(5)
memory usage: 37.0+ MB


In [72]:
apart['해제사유발생일'].value_counts()

20200727.0    44
20200714.0    41
20200720.0    37
20210326.0    35
20200715.0    32
              ..
20201220.0     1
20210220.0     1
20200809.0     1
20200705.0     1
20200919.0     1
Name: 해제사유발생일, Length: 359, dtype: int64

In [73]:
# 거래중 359건의 계약 해지가 발생한 것을 볼 수 있다.
# 해지된 계약은 아파트 값에 반영해야할지 의문이다. 

In [74]:
apart['시군구'].value_counts()

서울특별시 노원구 상계동      13558
서울특별시 노원구 중계동       8191
서울특별시 양천구 신정동       7051
서울특별시 구로구 구로동       6877
서울특별시 도봉구 창동        6423
                   ...  
서울특별시 종로구 신교동          2
서울특별시 종로구 신문로2가        2
서울특별시 마포구 구수동          1
서울특별시 종로구 수송동          1
서울특별시 종로구 연지동          1
Name: 시군구, Length: 335, dtype: int64

In [75]:
# 거래금액이 문자형으로 저장되어있고, 자리수 기호(,) 때문에 정수형으로 바꿔지지 않아서 자리수 기호르 삭제해주었다.
apart["거래금액(만원)"]=apart["거래금액(만원)"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
apart["거래금액(만원)"].head(3)

0    80000
1    85000
2    89900
Name: 거래금액(만원), dtype: object

In [76]:
# 문자형 변수를 수치형 변수로 변환해 주었다.
apart["거래금액(만원)"]=apart["거래금액(만원)"].astype("float")
apart["거래금액(만원)"].head(3)

0    80000.0
1    85000.0
2    89900.0
Name: 거래금액(만원), dtype: float64

In [77]:
# 크기(넓이)를 고려한 아파트 가격을 알아보기 위하여 1제곱미터 당 아파트 가격을 구해보도록 하겠다.
# 마지막 칼럼에 "거래금액(만원)/전용면적(㎡)" 이 추가되어 자동계산된다.
apart["거래금액(만원)/전용면적(㎡)"]=apart["거래금액(만원)"]/apart["전용면적(㎡)"]

In [78]:
# 이번 프로젝트를 "구" 단위로 데이터를 알아보기로 하였으므로 주소지를 기준으로 시군구( 실제로는 시구동으로 나뉘어짐 )를 분할한다.
apart[['시','구','동']] = apart.시군구.str.split(" ",expand=True)

In [79]:
apart
# 마지막 칼럼 4개를 보면 "거래금액(만원)/전용면적(㎡)","시","군","구" 가 추가 된것을 확인 할 수 있다.

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래금액(만원)/전용면적(㎡),시,구,동
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201703,15,80000.0,7,1988,언주로 103,NaN,1028.938907,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201704,19,85000.0,2,1988,언주로 103,NaN,1093.247588,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201707,8,89900.0,1,1988,언주로 103,NaN,1156.270096,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,201704,13,97500.0,4,1987,언주로 3,NaN,1219.207203,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,201704,17,98000.0,1,1987,언주로 3,NaN,1225.459547,서울특별시,강남구,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84221,서울특별시 중랑구 중화동,274-77,274.0,77.0,한영(104),67.57,202008,7,26000.0,2,2003,동일로144길 74,NaN,384.786148,서울특별시,중랑구,중화동
84222,서울특별시 중랑구 중화동,11,11.0,0.0,현대휴앤미,95.94,202007,10,44000.0,6,2007,봉화산로27가길 23,NaN,458.619971,서울특별시,중랑구,중화동
84223,서울특별시 중랑구 중화동,11,11.0,0.0,현대휴앤미,100.17,202012,3,54800.0,5,2007,봉화산로27가길 23,NaN,547.069981,서울특별시,중랑구,중화동
84224,서울특별시 중랑구 중화동,11-7,11.0,7.0,현대휴앤미(102동),77.71,202009,28,40000.0,5,2007,봉화산로27가길 19,NaN,514.734268,서울특별시,중랑구,중화동


In [80]:
seoul=apart["시"]

In [81]:
seoul.value_counts() # 결측치 확인 : 서울특별시가 아닌 다른 값으로 입련된 값이 있는지 확인

서울특별시    345993
Name: 시, dtype: int64

In [82]:
# 년도 별로 확인하기 위해 년도만 따로 뽑아냈다.
apart["년도"]=apart["계약년월"].astype("str")

In [83]:
apart["년도"]=apart["년도"].str[:4]

In [84]:
# 구별 평균 아파트 값을 보기 위해 필요한 칼럼만 추출
apart_gu = apart[["구","년도","거래금액(만원)/전용면적(㎡)","건축년도"]]

In [85]:
apart_gu

,구,년도,거래금액(만원)/전용면적(㎡),건축년도
0,강남구,2017,1028.938907,1988
1,강남구,2017,1093.247588,1988
2,강남구,2017,1156.270096,1988
3,강남구,2017,1219.207203,1987
4,강남구,2017,1225.459547,1987
...,...,...,...,...
84221,중랑구,2020,384.786148,2003
84222,중랑구,2020,458.619971,2007
84223,중랑구,2020,547.069981,2007
84224,중랑구,2020,514.734268,2007


In [86]:
# 구별 평균값을 구하기 위해 구 명(name)이 필요하였다.
# 수작업 없이 하고 싶었는데 아쉬운 점...
apart_gu["구"].value_counts()

노원구     34976
송파구     23197
강서구     20663
강남구     19263
강동구     18299
구로구     18094
성북구     18016
양천구     16096
도봉구     15607
서초구     14848
영등포구    14358
성동구     13631
동대문구    12940
마포구     12731
동작구     12492
은평구     11984
중랑구     11295
관악구     11218
서대문구    11049
강북구      7391
용산구      7221
광진구      6857
금천구      6276
중구       4531
종로구      2960
Name: 구, dtype: int64

In [87]:
# 구 명을 데이터 프레임으로 만들때 사용하기 위해 따로 리스트를 작성해두었다. 
seoul_gu=["노원구",
"송파구",
"강서구",
"강남구",
"강동구",
"구로구",
"성북구",
"양천구",
"도봉구",
"서초구",
"영등포구",
"성동구",
"동대문구",
"마포구",
"동작구",
"은평구",
"중랑구",
"관악구",
"서대문구",
"강북구",
"용산구",
"광진구",
"금천구",
"중구",
"종로구",]

In [89]:
# 데이터 프레임에서 바로 평균 값으로 병합하는 방법을 쓰고 싶었는데 잘 되지 않았다.(이부분은 꼭 해결을 해야할 것 같다.)
# 구별 평균 집값을 구하기 위해 반복문을 사용하였다.
# medv(Median Value)
medv=[]
for i in seoul_gu:
    medv.append(apart_gu[apart_gu["구"]==i]["거래금액(만원)/전용면적(㎡)"].mean())

In [90]:
medv

[668.3558863494111,
 1280.876405039977,
 795.7030279763378,
 1773.8226447685236,
 926.3039571622762,
 652.1481166080858,
 714.4136373346678,
 927.8422713975988,
 562.1695410167097,
 1547.5708457889245,
 928.8255972829458,
 1118.338008393614,
 775.8410417098669,
 1072.2134456769193,
 958.1129598042606,
 690.1804566890482,
 610.3512901000805,
 700.2845813687331,
 833.5804688698827,
 631.6176858136187,
 1274.0356888785086,
 996.5925528868042,
 590.9299462379819,
 936.6616401793168,
 871.5143279718943]

In [91]:
# 서울시 구명(이름)과 합쳐 2차원 배열로 만들었다. 
apartTotalmedv=[seoul_gu,medv]

In [92]:
# 2차원 배열로 만든 리스트를 데이터 프레임으로 반환하고, 쓰기 좋게 전치 시켜주었다. 
apartTotalmedv=pd.DataFrame(apartTotalmedv).T

In [93]:
apartTotalmedv

,0,1
0,노원구,668.355886
1,송파구,1280.876405
2,강서구,795.703028
3,강남구,1773.822645
4,강동구,926.303957
5,구로구,652.148117
6,성북구,714.413637
7,양천구,927.842271
8,도봉구,562.169541
9,서초구,1547.570846


In [94]:
# 칼럼명을 입력해주었다. 
apartTotalmedv.columns = ['구', '거래금액(만원)/전용면적(㎡)']

In [95]:
# 팀원들과 자료 공유를 위하여 csv로 저장하였다. 경로를 잘못입력 한부분이 있는데 이부분은 수정하였다. 
# apartTotalmedv
apartTotalmedv.to_csv("./fin_data/apartTotalmedv.csv",header=True,index=False)

In [96]:
# 수작업 2 밑에 나머지 코딩은 17년도부터 20년도까지 구별 아파트 평균 집값을 구하기 위한 과정이고
# 위에 전체 기간 평균 아파트 가격을 구하는 코딩과 별 다를 것 없다. 
# 이 모든 과정을 한번에 할 파이프 라인을 꼭 만들어 봐야하겠다. 
apart2017=apart_gu[apart_gu["년도"]=="2017"]
apart2018=apart_gu[apart_gu["년도"]=="2018"]
apart2019=apart_gu[apart_gu["년도"]=="2019"]
apart2020=apart_gu[apart_gu["년도"]=="2020"]

In [97]:
medv2017=[]
for i in seoul_gu:
    medv2017.append(apart2017[apart2017["구"]==i]["거래금액(만원)/전용면적(㎡)"].mean())

In [98]:
medv2017

[541.1091222913183,
 1029.9145237429166,
 652.2719839442088,
 1466.0823194668621,
 746.5778891032552,
 543.7846052312177,
 576.890195671003,
 777.9962031620374,
 468.8830988197136,
 1238.216487048515,
 717.6506118667578,
 829.1752296832866,
 610.6085603296561,
 835.2820449299854,
 746.2936111113003,
 562.8011306311403,
 498.25494715295895,
 572.5300409284267,
 662.2876109994726,
 514.4329777499834,
 1012.6406049236574,
 791.1224267329189,
 473.8227098981608,
 761.2215290699108,
 696.5950572358337]

In [102]:
df2017=[seoul_gu,medv2017]

In [104]:
apart2017medv=pd.DataFrame(df2017).T

In [105]:
apart2017medv.columns = ['구', '거래금액(만원)/전용면적(㎡)']

In [119]:
apart2017medv.to_csv("./fin_data/apart2017medv.csv",header=True,index=False)
# apart2017medv


In [108]:
medv2018=[]
for i in seoul_gu:
    medv2018.append(apart2018[apart2018["구"]==i]["거래금액(만원)/전용면적(㎡)"].mean())

In [109]:
df2018=[seoul_gu,medv2018]

In [110]:
apart2018medv=pd.DataFrame(df2018)

In [111]:
apart2018medv=apart2018medv.T

In [116]:
apart2018medv.columns = ['구', '거래금액(만원)/전용면적(㎡)']

In [120]:
# apart2018medv

In [122]:
apart2018medv.to_csv("./fin_data/apart2018medv.csv",header=True,index=False)

In [123]:
medv2019=[]
for i in seoul_gu:
    medv2019.append(apart2019[apart2019["구"]==i]["거래금액(만원)/전용면적(㎡)"].mean())
df2019=pd.DataFrame([seoul_gu,medv2019]).T
df2019.columns=["구","거래금액(만원)/전용면적(㎡)"]
df2019.to_csv("apart2019medv.csv",header=True,index=False)

In [124]:
medv2020=[]
for i in seoul_gu:
    medv2020.append(apart2020[apart2020["구"]==i]["거래금액(만원)/전용면적(㎡)"].mean())
df2020=pd.DataFrame([seoul_gu,medv2020]).T
df2020.columns=["구","거래금액(만원)/전용면적(㎡)"]
df2020.to_csv("apart2020medv.csv",header=True,index=False)

In [125]:
# pipeline 만들기
# 11,12,13,14,15,16,17,18,19,20,21 까지 자동으로 다운로드 받아주고 
# 구별로 평균을 내주는 파이프라인을 만들 수 있을까
medv+11="paris"

SyntaxError: cannot assign to operator (<ipython-input-125-f54a9070f6b6>, line 4)

In [126]:
medv11

NameError: name 'medv11' is not defined